In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pheochromocytoma_and_Paraganglioma"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Find the directory corresponding to Pheochromocytoma_and_Paraganglioma
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Find the directory that matches our trait: Pheochromocytoma_and_Paraganglioma
matching_dirs = [dir_name for dir_name in tcga_dirs 
                if "pheochromocytoma" in dir_name.lower() or "paraganglioma" in dir_name.lower()]

if not matching_dirs:
    print(f"No matching directory found for trait: {trait}")
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
else:
    # Select the most relevant directory
    selected_dir = matching_dirs[0]  # Should be 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)'
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'ct_scan', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'disease_detected_on_screening', 'eastern_cancer_oncology_group', 'form_completion_date', 'gender', 'histological_type', 'history_of_neoadjuvant_treatment', 'history_pheo_or_para_anatomic_site', 'history_pheo_or_para_include_benign', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'initial_weight', 'is_ffpe', 'karnofsky_performance_score', 'laterality', 'lost_follow_up', 'lymph_node_examined_count', 'new_neoplasm_confirmed_diagnosis_method_name', 'new_neoplasm_event_occurrence_anatomic_site', 'new_neoplasm_event_type', 'new_neoplasm_occurrence_anatomic_site_text', 'new_tumor

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# First, we need to find the correct directory for the trait
# List all directories in the TCGA root dir to find the one for Pheochromocytoma and Paraganglioma
tcga_dirs = os.listdir(tcga_root_dir)

# Find directories that might correspond to Pheochromocytoma and Paraganglioma
# Common abbreviation for this trait is PCPG
cohort_dir = None
for dir_name in tcga_dirs:
    if "PCPG" in dir_name or "Pheochromocytoma" in dir_name or "Paraganglioma" in dir_name:
        cohort_dir = os.path.join(tcga_root_dir, dir_name)
        break

if cohort_dir is None:
    # If no matching directory is found, print available directories
    print(f"Available directories in {tcga_root_dir}:")
    print(tcga_dirs)
    raise FileNotFoundError(f"Could not find directory for {trait} in {tcga_root_dir}")
else:
    print(f"Found cohort directory: {cohort_dir}")

# Get the clinical file path
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')

# Preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)

# Preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("\nGender columns preview:")
print(gender_preview)


Found cohort directory: ../../input/TCGA/TCGA_Pheochromocytoma_Paraganglioma_(PCPG)
Age columns preview:
{'age_at_initial_pathologic_diagnosis': [78, 21, 21, 48, 48], 'days_to_birth': [-28497, -7834, -7834, -17790, -17790]}

Gender columns preview:
{'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine the age columns
print("Examining age columns...")
for col_name, values in {'age_at_initial_pathologic_diagnosis': [78, 21, 21, 48, 48], 
                         'days_to_birth': [-28497, -7834, -7834, -17790, -17790]}.items():
    print(f"{col_name}: {values}")
    print(f"  Missing values: {values.count(None) if None in values else 0}")
    print(f"  Data type: {type(values[0])}")

# Examine the gender columns
print("\nExamining gender columns...")
for col_name, values in {'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'MALE', 'MALE']}.items():
    print(f"{col_name}: {values}")
    print(f"  Missing values: {values.count(None) if None in values else 0}")
    print(f"  Unique values: {set(values)}")

# Select appropriate columns
age_col = 'age_at_initial_pathologic_diagnosis'  # Direct age values in years
gender_col = 'gender'  # Clear gender labels

print("\nChosen demographic columns:")
print(f"  Age column: {age_col}")
print(f"  Gender column: {gender_col}")


Examining age columns...
age_at_initial_pathologic_diagnosis: [78, 21, 21, 48, 48]
  Missing values: 0
  Data type: <class 'int'>
days_to_birth: [-28497, -7834, -7834, -17790, -17790]
  Missing values: 0
  Data type: <class 'int'>

Examining gender columns...
gender: ['FEMALE', 'FEMALE', 'FEMALE', 'MALE', 'MALE']
  Missing values: 0
  Unique values: {'FEMALE', 'MALE'}

Chosen demographic columns:
  Age column: age_at_initial_pathologic_diagnosis
  Gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Find matching directory for Pheochromocytoma_and_Paraganglioma
matching_dirs = [dir_name for dir_name in os.listdir(tcga_root_dir) 
                if "pheochromocytoma" in dir_name.lower() or "paraganglioma" in dir_name.lower()]
selected_dir = matching_dirs[0]  # Should find 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)'
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for pheochromocytoma and paraganglioma patients."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 187 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (187, 19851) (samples x features)


After handling missing values, data shape: (187, 19851)
For the feature 'Pheochromocytoma_and_Paraganglioma', the least common label is '0' with 3 occurrences. This represents 1.60% of the dataset.
The distribution of the feature 'Pheochromocytoma_and_Paraganglioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 35.0
  50% (Median): 46.0
  75%: 57.5
Min: 19
Max: 83
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 84 occurrences. This represents 44.92% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset was determined to be unusable and was not saved.
